In [3]:
import __init__

Note for this study its important that the Material ID is in the base df

In [15]:
import pandas as pd
df = pd.read_parquet('/home/cyprien/Data_Gen/mp-20-data/all_data.parquet')
df

,Material ID,Reduced_Formula,CIF,Split
0,mp-1221227,Na3MnCoNiO6,# generated using pymatgen\ndata_Na3MnCoNiO6\n...,train
1,mp-974729,Nd(Al2Cu)4,# generated using pymatgen\ndata_Nd(Al2Cu)4\n_...,train
2,mp-1185360,LiMnIr2,# generated using pymatgen\ndata_LiMnIr2\n_sym...,train
3,mp-1188861,LiCSN,# generated using pymatgen\ndata_LiCSN\n_symme...,train
4,mp-677272,La2EuS4,# generated using pymatgen\ndata_La2EuS4\n_sym...,train
...,...,...,...,...
45224,mp-21084,In6Ga2PtO8,# generated using pymatgen\ndata_In6Ga2PtO8\n_...,test
45225,mp-571486,CuSe,# generated using pymatgen\ndata_CuSe\n_symmet...,test
45226,mp-14410,Tl6TeO12,# generated using pymatgen\ndata_Tl6TeO12\n_sy...,test
45227,mp-1079192,Sr2GdRuO6,# generated using pymatgen\ndata_Sr2GdRuO6\n_s...,test


In [13]:
!python _utils/_preprocessing/_calculate_XRD.py \
    --input_parquet /home/cyprien/Data_Gen/mp-20-data/all_data.parquet \
    --output_parquet HF-databases/mp_20_pxrd/mp_20_pxrd.parquet \
    --num_workers 16

Loading database from /home/cyprien/Data_Gen/mp-20-data/all_data.parquet
Loaded 45229 entries
Processing XRD patterns for the entries
Generating XRD patterns: 100%|███| 45229/45229 [03:02<00:00, 247.50structures/s]
Computing condition vectors from XRD patterns
Processing XRD patterns: 100%|███| 45229/45229 [00:00<00:00, 45505.59patterns/s]
Theta range: 0-90, Intensity range: 0-100
Saving results to HF-databases/mp_20_pxrd/mp_20_pxrd.parquet
done


In [ ]:
!python _utils/_preprocessing/_cleaning.py \
    --input_parquet HF-databases/mp_20_pxrd/mp_20_pxrd.parquet \
    --output_parquet HF-databases/mp_20_pxrd/mp_20_pxrd_clean.parquet \
    --property_columns "['Condition Vector']" \
    --num_workers 16 \
    --filter_to 1024

Loading data from HF-databases/mp_20_pxrd/mp_20_pxrd.parquet as Parquet with zstd compression...

Normalizing property columns

Lets augment the CIFs now (parallelizing sometimes takes a min before speeding up
Number of CIFs before preprocessing: 45229
Number of workers: 16
100%|███████████████████████████████████| 45229/45229 [00:11<00:00, 3997.55it/s]
Number of CIFs before filtering out bad ones:  45229
Number of CIFs after filtering: 45229

Saving updated dataframe to HF-databases/mp_20_pxrd/mp_20_pxrd_clean.parquet as Parquet with zstd compression...
Preprocessing completed successfully.


In [20]:
!python _utils/_preprocessing/_save_dataset_to_HF.py \
    --input_parquet HF-databases/mp_20_pxrd/mp_20_pxrd_clean.parquet \
    --output_parquet HF-databases/mp_20_pxrd/mp_20_pxrd.parquet \
    --save_hub

Loading Hugging Face API key from API_keys.jsonc
Loading data from HF-databases/mp_20_pxrd/mp_20_pxrd_clean.parquet as Parquet with zstd compression
Splitting dataset according to the 'Split' column
Train columns: ['Material ID', 'Reduced_Formula', 'CIF', 'Condition Vector']
Uploading the dataset shards: 100%|███████████████| 1/1 [00:00<00:00,  1.26it/s]
Dataset saved to Hugging Face Hub as c-bone/mp_20_pxrd


In [ ]:
!python _utils/_generating/make_prompts.py \
    --HF_dataset 'c-bone/mp_20_pxrd' \
    --split 'test' \
    --automatic \
    --output_parquet '_artifacts/mp-20-pxrd/mp-test_prompts.parquet' \
    --level 'level_3' \
    --condition_columns 'Condition Vector'
    # --remove_ref_columns

Loaded 9045 records from c-bone/mp_20_pxrd (test split)

Generated automatic prompts at level_3
Using condition columns: ['Condition Vector']

First 3 rows:
Material ID Reduced_Formula                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

# Benchmark with a model trained from scratch

In [ ]:
!python _utils/_generating/generate_CIFs.py --config '_config_files/generation/conditional/xrd_studies/mp-xrd-scratch_eval.jsonc'

In [ ]:
!python _utils/_generating/postprocess.py \
    --input_parquet '_artifacts/mp-20-pxrd/mp-scratch-20perp-test_gen.parquet' \
    --output_parquet '_artifacts/mp-20-pxrd/mp-scratch-20perp-test_post.parquet' \
    --num_workers 32 \
    --column_name 'Generated CIF'

In [ ]:
!python _utils/_metrics/XRD_metrics.py \
    --input_parquet '_artifacts/mp-20-pxrd/mp-scratch-20perp-test_post.parquet' \
    --num_gens 20 \
    --ref_parquet '_artifacts/mp-20-pxrd/mp-test_ref.parquet' \
    --output_parquet '_artifacts/mp-20-pxrd/mp-scratch-20perp-test_metrics.parquet' \
    --num_workers 16 \
    --validity_check 'diffcsp'

In [4]:
!python _utils/_metrics/XRD_metrics.py \
    --input_parquet '_artifacts/mp-20-pxrd/mp-scratch-20perp-test_post.parquet' \
    --num_gens 1 \
    --ref_parquet '_artifacts/mp-20-pxrd/mp-test_ref.parquet' \
    --output_parquet '_artifacts/mp-20-pxrd/mp-scratch-1perp-nosmact-test_metrics.parquet' \
    --num_workers 24 \
    --validity_check 'none'

Using 1 generation(s) per compound
Using 16 workers for parallel processing (based on input size)
Using rank=1 rows for num_gens=1 (rank column detected)
Loaded 9016 materials from _artifacts/mp-20-pxrd/mp-scratch-20perp-test_post.parquet
Using 9016 matched materials from test DB
Parsing true CIFs: 100%|███████████████████| 9016/9016 [00:53<00:00, 167.95it/s]
Processing 9016 CIFs across 9016 materials
Parsing and sensible check for gen CIFs: 100%|█| 9016/9016 [00:06<00:00, 1314.92
Materials processed: 9016
Materials with sensible structures: 9016
Comparing structures:  74%|███████████▉    | 6700/9016 [01:02<00:18, 126.35it/s]spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and

In [ ]:
!python _utils/_metrics/XRD_metrics.py \
    --input_parquet '_artifacts/mp-20-pxrd/mp-scratch-20perp-test_post.parquet' \
    --num_gens 1 \
    --ref_parquet '_artifacts/mp-20-pxrd/mp-test_ref.parquet' \
    --output_parquet '_artifacts/mp-20-pxrd/mp-scratch-1rand-test_metrics.parquet' \
    --num_workers 24 \
    --validity_check 'diffcsp' \
    --sort_gens 'random'

In [1]:
import __init__
import pandas as pd
from _utils import get_metrics_xrd

df_metrics = pd.read_parquet('_artifacts/mp-20-pxrd/mp-scratch-20perp-test_metrics.parquet')
metrics = get_metrics_xrd(df_metrics, n_test=9046, only_matched=True)
print("\n\n")
df_metrics = pd.read_parquet('_artifacts/mp-20-pxrd/mp-scratch-1perp-test_metrics.parquet')
metrics = get_metrics_xrd(df_metrics, n_test=9046, only_matched=True)
print("\n\n")
df_metrics = pd.read_parquet('_artifacts/mp-20-pxrd/mp-scratch-1perp-nosmact-test_metrics.parquet')
metrics = get_metrics_xrd(df_metrics, n_test=9046, only_matched=True)


Navigated to package root: /home/cyprien/CrystaLLMv2_PKV
Added package root to Python path
Computing metrics only on matched structures (6821 entries)
Number of matched structures: 6821 / 9046
Mean RMS-d: 0.0471
Percent Matched (%): 75.40% (6821/9046)
a MAE: 0.2228
b MAE: 0.2183
c MAE: 0.3572
Volume MAE: 22.0161
a R^2: 0.9509
b R^2: 0.9647
c R^2: 0.9838
Volume R^2: 0.9882
Average Score: 1.1524



Computing metrics only on matched structures (5675 entries)
Number of matched structures: 5675 / 9046
Mean RMS-d: 0.0444
Percent Matched (%): 62.73% (5675/9046)
a MAE: 0.1725
b MAE: 0.1746
c MAE: 0.3010
Volume MAE: 19.6953
a R^2: 0.9724
b R^2: 0.9806
c R^2: 0.9864
Volume R^2: 0.9917
Average Score: 1.1153



Computing metrics only on matched structures (6243 entries)
Number of matched structures: 6243 / 9046
Mean RMS-d: 0.0465
Percent Matched (%): 69.01% (6243/9046)
a MAE: 0.1696
b MAE: 0.1740
c MAE: 0.3000
Volume MAE: 19.6144
a R^2: 0.9738
b R^2: 0.9811
c R^2: 0.9881
Volume R^2: 0.9915
Average